In [1]:
import pandas as pd

In [2]:
import requests

In [3]:
# zum Zusammenbauen der Bibsonomy URL brauchen wir alles, was vor und nach der Query kommt
url_base = "https://www.bibsonomy.org/json/search/"
url_attr = "?items=100"

In [4]:
# ein kurzer Test: leerzeichen müssen ersetzt werden!
requests.get(url_base + "computer science".replace(" ", "%20") + url_attr).json().keys()

dict_keys(['types', 'properties', 'items'])

In [5]:
def df_from_query(query):
    # die URL; wir müssen Leerzeichen ersetzen mit "%20" nach ASCII Encoding guideline
    url = url_base + query.replace(" ", "%20") + url_attr
    
    result = requests.get(url)
    json_data = result.json()
    
    # im Feld "items" stecken die relevanten Dateneinträge
    dataframe = pd.DataFrame(json_data["items"])

    # wir filtern so, dass nur Publikationen bleiben
    dataframe.loc[dataframe["type"] == "Publication"]

    # wir möchten die query in die Daten eintragen 
    dataframe["query"] = query
    
    # die Rückgabe der Methode nicht vergessen:
    return dataframe
    

In [6]:
# kurzer Test:
df_cs = df_from_query("computer science")

In [7]:
# wir erstellen eine leere Liste, in die wir unsere Dataframes packen
dataframes = []

# eine Liste von Suchanfragen
queries = ["cheese", "water management", "libraries", "Europe", "politics"]

# jetzt gehen wir die Liste von Suchanfragen durch und rufen unsere Funktion für jede Suchanfrage auf
for query in queries:
    df = df_from_query(query)
    # schließlich fügen wir das DataFrame über append unserer Liste von Dataframes hinzu
    dataframes.append(df)

    # wie wir uns eine Statusausgabe schreiben würden:
    # print("Ergebnisse für ", query , " gefunden. Dataframe enthält ",len(df), " Einträge.")
    
    # mit einem f String lassen sich Variablen in Strings viel einfacher formatieren:
    print(f"{len(df)} Ergebnisse für query {query} gefunden.")
    
    

200 Ergebnisse für query cheese gefunden.
146 Ergebnisse für query water management gefunden.
200 Ergebnisse für query libraries gefunden.
200 Ergebnisse für query Europe gefunden.
200 Ergebnisse für query politics gefunden.


In [8]:
# Da die dataframes die selbe From (column titles sind identisch, wenn für alle vorhanden) haben, können wir die Dataframes über concat zusammenfügen
data = pd.concat(dataframes, ignore_index=True)

In [9]:
data["query"]

0        cheese
1        cheese
2        cheese
3        cheese
4        cheese
         ...   
941    politics
942    politics
943    politics
944    politics
945    politics
Name: query, Length: 946, dtype: object

In [10]:
data.to_json("data/json_data.json")

In [11]:
df_cs.columns

Index(['type', 'id', 'tags', 'intraHash', 'label', 'user', 'description',
       'date', 'changeDate', 'count', 'url', 'interHash', 'pub-type',
       'journal', 'publisher', 'address', 'year', 'author', 'authors',
       'volume', 'number', 'pages', 'abstract', 'issn', 'doi', 'bibtexKey',
       'citeulike-article-id', 'citeulike-linkout-0', 'isbn',
       'citeulike-linkout-2', 'citeulike-linkout-1', 'citeulike-linkout-4',
       'citeulike-linkout-3', 'priority', 'posted-at', 'booktitle', 'series',
       'location', 'acmid', 'numpages', 'note', 'editor', 'editors', 'keyword',
       'affiliation', 'issue', 'issue_date', 'ean', 'asin', 'dewey', 'eprint',
       'language', 'refid', 'timestamp', 'username', 'intrahash', 'interhash',
       'groups', 'owner', 'bibsource', 'ee', 'publisher_address', 'date-added',
       'bdsk-url-1', 'w-type', 'w-projects', 'date-modified', 'urldate',
       'query'],
      dtype='object')

In [12]:
df_test = df_cs[['type', 'url', 'year', 'author', 'authors', 'abstract', 'doi', 'query']]

In [13]:
df_test = df_test.drop(["doi", "abstract"], axis="columns")

In [14]:
df_test

,type,url,year,author,authors,query
0,Bookmark,https://ocw.mit.edu/courses/electrical-enginee...,NaN,NaN,NaN,computer science
1,Bookmark,https://abakcus.com/book_genre/computer-science/,NaN,NaN,NaN,computer science
2,Bookmark,https://www.slideshare.net/BartRienties/educat...,NaN,NaN,NaN,computer science
3,Bookmark,http://www.orb-academic.org/index.php/journal-...,NaN,NaN,NaN,computer science
4,Bookmark,http://ocw.mit.edu/courses/electrical-engineer...,NaN,NaN,NaN,computer science
...,...,...,...,...,...,...
195,Publication,http://dx.doi.org/10.1145/2543882.2543886,2013,"[Ari Korhonen, Thomas L. Naps, Charles Boisver...","[{'first': 'Ari', 'last': 'Korhonen'}, {'first...",computer science
196,Publication,http://dx.doi.org/10.1145/2543882.2543886,2013,"[Ari Korhonen, Thomas L. Naps, Charles Boisver...","[{'first': 'Ari', 'last': 'Korhonen'}, {'first...",computer science
197,Publication,http://dblp.uni-trier.de/db/journals/csedu/cse...,2006,"[Michal Armoni, Judith Gal-Ezer, Orit Hazzan]","[{'first': 'Michal', 'last': 'Armoni'}, {'firs...",computer science
198,Publication,http://dblp.uni-trier.de/db/journals/csedu/cse...,1990,"[Cathleen A. Norris, James L. Poirot, Gerald K...","[{'first': 'Cathleen A.', 'last': 'Norris'}, {...",computer science


# Fragen
### Allgemein
- Ihr geht sicher noch einmal darauf ein, wie die in der Teilaufgabe erstellt json-Datei für die weitere Bearbeitung aussehen soll? (Größe? Ich habe NaN-Einträge. Ist das ein ko-Kriterium?) --> Wolfgang
- Ich habe totale Probleme bei GitHub, die ich nicht lösen kann. Kann ich mich mit Bitte um einen 'Crashkurs' am 24.1. an Dich wenden? --> Wolfgang

### Python & Pandas
- .apply funktioniert. Mir ist aber nicht ganz klar, wie?, dass. beim Export in json.
- Welche Möglichkeiten gibt es, um Spalten aus dem DataFrame zu löschen? Mit .drop kann auf den Spaltennamen zugegriffen und gelöscht werden. Und wenn dies nicht möglich ist, weil z.B. ein Spaltenname doppelt vergeben ist? Gibt es einen eleganteren Weg, als eine Kopie mit den gewünschten Spalten zu öffnen und damit die Variable zu überschreiben? Etwa über einen Spaltenindex?

In [15]:
df_lamp = df_from_query("desk lamp")

In [16]:
df_lamp.columns

Index(['type', 'id', 'tags', 'intraHash', 'label', 'user', 'description',
       'date', 'changeDate', 'count', 'url', 'interHash', 'pub-type',
       'journal', 'year', 'author', 'authors', 'volume', 'number', 'pages',
       'ee', 'bibtexKey', 'booktitle', 'series', 'publisher', 'editor',
       'editors', 'isbn', 'dnbtitleid', 'address', 'refid', 'abstract',
       'holdings', 'publisher_address', 'size', 'notes', 'eventtitle',
       'language', 'doi', 'issn', 'citeulike-article-id',
       'citeulike-linkout-1', 'priority', 'posted-at', 'eprint',
       'citeulike-linkout-0', 'archiveprefix', 'query'],
      dtype='object')

In [18]:
# Über eine liste von einer liste von column names (Doppelte [[]] !) können wir spalten auswählen
df = df_lamp[['type', 'url', 'year', 'author', 'authors', 'abstract', 'doi', 'query']]

In [19]:
df

,type,url,year,author,authors,abstract,doi,query
0,Bookmark,http://www.heise.de/tp/artikel/27/27814/1.html,NaN,NaN,NaN,NaN,NaN,desk lamp
1,Bookmark,http://www.christian-von-kamp.de/,NaN,NaN,NaN,NaN,NaN,desk lamp
2,Bookmark,http://www.lapdawg.com/functions/lap-writing-d...,NaN,NaN,NaN,NaN,NaN,desk lamp
3,Bookmark,http://www.securityfocus.com/cgi-bin/index.cgi...,NaN,NaN,NaN,NaN,NaN,desk lamp
4,Bookmark,http://www.securityfocus.com/cgi-bin/index.cgi...,NaN,NaN,NaN,NaN,NaN,desk lamp
...,...,...,...,...,...,...,...,...
114,Publication,,1998,[Sodikdjon Kodirov],"[{'first': 'Sodikdjon', 'last': 'Kodirov'}]",NaN,NaN,desk lamp
115,Publication,,2012,[Katharina Hubertus],"[{'first': 'Katharina', 'last': 'Hubertus'}]",NaN,NaN,desk lamp
116,Publication,,1988,[Olaf Schneewind],"[{'first': 'Olaf', 'last': 'Schneewind'}]",NaN,NaN,desk lamp
117,Publication,,2005,[Birgit Gehrisch],"[{'first': 'Birgit', 'last': 'Gehrisch'}]",NaN,NaN,desk lamp


In [20]:
# mit drop können wir eine Liste von columns entfernen
df = df.drop(["doi", "abstract"], axis="columns")

In [21]:
## Apply
import random
df_test = pd.DataFrame([[5500, 200],[7000, 350]], columns=["Strecke","Steigung"], index=["Peter", "Petra"] )
df_test["Tag"] = "Sonntag"

In [22]:
df_test

,Strecke,Steigung,Tag
Peter,5500,200,Sonntag
Petra,7000,350,Sonntag


In [23]:
def to_km(meters):
    return meters/1000

In [29]:
# so bekommen wir einen Fehler - wir möchten eigentlich ja auch nur die Strecke umwandeln!
df_test.apply(to_km)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [30]:
df_test

,Strecke,Steigung,Tag,Strecke in km
Peter,5500,200,Sonntag,5.5
Petra,7000,350,Sonntag,7.0


In [31]:
df_test["Strecke"].apply(to_km)

Peter    5.5
Petra    7.0
Name: Strecke, dtype: float64

In [32]:
# apply mit lambda gibt ein Series Object zurück
df_test.apply(lambda row: to_km(row["Strecke"]), axis=1)

Peter    5.5
Petra    7.0
dtype: float64

In [36]:
# wir speichern in eine Spalte
df_test["Strecke in km"] = df_test.apply(lambda row: to_km(row["Strecke"]), axis=1)

In [34]:
df_test

,Strecke,Steigung,Tag,Strecke in km
Peter,5500,200,Sonntag,5.5
Petra,7000,350,Sonntag,7.0


In [37]:
# Anwendung mit mehreren columns

df_test.apply(lambda row: row["Steigung"]/row["Strecke in km"], axis=1)

Peter    36.363636
Petra    50.000000
dtype: float64